In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import openpyxl
import os
from cloudmanufacturing.data import create_distance_matrix
from cloudmanufacturing.data import create_excel_table
from cloudmanufacturing.data import solve_excel
from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.mip_solver import mip_solve
from cloudmanufacturing.validation import objvalue, construct_delta
from cloudmanufacturing.graph import dglgraph_fixed, graph_gamma, os_type, ss_type, so_type, calculate_optimal
import logging
import pickle

## Генерация тренировочных данных

### Для генерации данных нужно задать количество задач N в ячейке ниже
 
Число городов, задачи и подзадач берутся из диапазона 3-25

In [ ]:
# лучше брать больше 80
N = 1000
stop_count = 100

In [ ]:
# для логирования
logger = logging.getLogger("../data/TrainDataGeneration")
logger.setLevel(logging.INFO)

# create the logging file handler
fh = logging.FileHandler("../data/TrainDataGeneration.log")

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)

# add handler to logger object
logger.addHandler(fh)
sheets = []
try:
    
	trios = []
	for i in range(N):
		trios.append(tuple((random.randint(3,25),random.randint(3,25),random.randint(3,25))))
	trios.sort()

	file_path = '../data/train_data.xlsx'
	wb = openpyxl.Workbook()
	wb.save(file_path)
    
    
	for n_operations, n_suboperations, n_cities in trios:
		sheets.append(create_excel_table(n_operations, n_suboperations, n_cities, n_problem=1, filepath=file_path ))
	logger.info(f"Created {len(sheets)} train tasks and saved in excel table") 
 
except:
	logger.exception("Train tasks not created")


## Расчет оптимальных решений на тренировочных данных

В результате будет созданы файлы в папке data:

  1. Логи - TrainDataGeneration.log
  2. Сгенерированный данные в формате fatahi - train_data.xlsx
  3. Решения в том числе неоптимальные - train_data_solved.xlsx
  4. Бинарный файл с DGL графами для оптимальеных решений - train_data_solve.pickle
  5. Бинарный файл с названиями листов - train_data_sheet_names.pickle
  6. Тренировочные данные в формате fatahi, включающие те данных, для которых найдены оптимальные решения-  train_data_OPTIMAL.xlsx

In [ ]:
train_pth = '../data/train_data.xlsx'
train_dataset = read_fatahi_dataset(train_pth, sheets)

In [ ]:
try:
	logger.info('Start calculating train dataset')
	calculate_optimal(train_pth, logger, train_dataset, os_type, ss_type,stop_count)
   	 
except:
	logger.exception("Failed data solving")

In [ ]:
fatahi_pth = '../data/fatahi.xlsx'
dataset_fatahi = read_fatahi_dataset(fatahi_pth)

In [ ]:
try:
	logger.info('Start calculating fatahi dataset')
	calculate_optimal(fatahi_pth, logger, dataset_fatahi, os_type, ss_type)
	logger.info('End fatahi dataset')
except:
	logger.exception("Failed fatahi data solving")